In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
import os

BASE_DIR = os.path.abspath("")
CSV_PATH = os.path.join(BASE_DIR, "data", "sale.csv")

In [2]:
df = pd.read_csv(CSV_PATH)
df.head()

C:\Users\aless\AppData\Local\Temp\ipykernel_25368\2186986378.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


,regione,citta,quartiere,prezzo,datetime,posti auto,bagni per stanza,bagni,stanze,ultimo piano,...,giardino privato,impianto allarme,portiere,piscina,villa,intera proprieta,appartamento,attico,loft,mansarda
0,sardegna,Valledoria,Via Alessandro Volta,78000.0,2022-12-17,0,0.333333,1.0,3.0,0,...,0,0,0,0,0,0,1,0,0,0
1,sardegna,San Teodoro,NaN,460000.0,2023-03-31,1,0.666667,2.0,3.0,1,...,0,0,1,1,0,0,1,0,0,0
2,sardegna,Cagliari,Fiera - Monte Mixi,387000.0,2023-04-18,0,0.500000,2.0,4.0,0,...,0,0,0,0,0,0,1,0,0,0
3,sardegna,Cagliari,Piazza Giovanni XXIII,257000.0,2023-04-14,0,0.400000,2.0,5.0,0,...,0,0,0,0,0,0,1,0,0,0
4,sardegna,Budoni,Località Baia Sant'Anna,370000.0,2023-03-31,1,0.666667,2.0,3.0,0,...,1,0,0,0,1,0,0,0,0,0


In [3]:
df.count().sort_values(ascending= False)

regione                        37087
datetime                       37087
posti auto                     37087
balcone                        37087
ultimo piano                   37087
arredato                       37087
villa                          37087
piscina                        37087
portiere                       37087
impianto allarme               37087
giardino privato               37087
giardino comune                37087
cantina                        37087
cancello elettrico             37087
fibra ottica                   37087
esposizione esterna            37087
impianto tv                    37087
appartamento                   37087
attico                         37087
loft                           37087
intera proprieta               37087
mansarda                       37087
vista mare                     37082
citta                          37080
bagni                          36480
prezzo                         35534
quartiere                      35154
s

In [4]:
df["prezzo"].describe()

count    3.553400e+04
mean     4.289680e+31
std      2.383630e+33
min      1.000000e+00
25%      7.900000e+04
50%      2.390000e+05
75%      4.620000e+05
max      1.950000e+35
Name: prezzo, dtype: float64

In [5]:
# Dato che ci sono valori impossibilmente grandi li rimouvo:
Q1 = df["prezzo"].quantile(0.25)
Q3 = df["prezzo"].quantile(0.75)
IQR = Q3 - Q1

df = df[(df["prezzo"] >max(Q1 - 1.5*IQR, 50000)) & (df["prezzo"] < Q3 + 1.5*IQR)]

In [6]:
# Rimuovo tutte le righe che hanno Nan nella mia variabile target:
df = df.dropna(subset= ["prezzo"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23232 entries, 0 to 37085
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   regione                      23232 non-null  object 
 1   citta                        23226 non-null  object 
 2   quartiere                    22084 non-null  object 
 3   prezzo                       23232 non-null  float64
 4   datetime                     23232 non-null  object 
 5   posti auto                   23232 non-null  int64  
 6   bagni per stanza             18805 non-null  float64
 7   bagni                        22981 non-null  float64
 8   stanze                       18909 non-null  float64
 9   ultimo piano                 23232 non-null  int64  
 10  stato                        3 non-null      object 
 11  classe energetica            3 non-null      object 
 12  vista mare                   23232 non-null  float64
 13  riscaldamento central

In [7]:
# Cerco i valori piu strettamente correlati al prezzo e quelli che ha senso considerare:
corr_matrix = df.corr(numeric_only= True, method= "spearman")

check_value = 0.2

strong_corr = corr_matrix[(corr_matrix >= check_value)|(corr_matrix <= - check_value)]
strong_corr_target = strong_corr["prezzo"].dropna().sort_values(ascending= False)
column_variables = strong_corr_target.head(15).index
column_variables = column_variables.tolist()
column_variables.append("regione")
column_variables.append("citta")
column_variables.append("superficie/stanze")
df["superficie/stanze"] = df["superficie"] / df["stanze"]
column_variables

['prezzo',
 'bagni',
 'superficie',
 'stanze',
 'portiere',
 'bagni per stanza',
 'villa',
 'appartamento',
 'regione',
 'citta',
 'superficie/stanze']

In [8]:
# Creo il modello usando una pipeline:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

column_variables.remove("prezzo")

cat_col = ["citta", "regione"]
num_col = [col for col in column_variables if col not in cat_col]

num_tfr = SimpleImputer(strategy= "mean")

cat_tfr = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy= "most_frequent")),
    ("encoder", OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value= -1))
])

regressor = RandomForestRegressor(n_estimators= 200, random_state= 3)

preprocessor = ColumnTransformer(transformers=[
    ("num", num_tfr, num_col),
    ("cat", cat_tfr, cat_col)
])

my_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", regressor)
])

In [9]:
df["prezzo"].describe()

count    2.323200e+04
mean     3.280921e+05
std      2.056592e+05
min      5.060000e+04
25%      1.760000e+05
50%      2.750000e+05
75%      4.200000e+05
max      1.030000e+06
Name: prezzo, dtype: float64

In [10]:
X = df[column_variables]
y = df["prezzo"]

param_grid = {
    "regressor__n_estimators" : [550],
    "regressor__max_depth" : [30],
    "regressor__min_samples_split" : [2],
    "regressor__min_samples_leaf" : [1]
}

grid_search = GridSearchCV(
    estimator = my_pipeline,
    param_grid = param_grid,
    cv = 5,
    scoring = "neg_mean_squared_error",
    n_jobs = -1
)

grid_search.fit(X, y)

RMSE = np.sqrt(-grid_search.best_score_)

In [11]:
RMSE_percentage = RMSE/df["prezzo"].mean() * 100
float(RMSE_percentage)

35.297404799101464

Sono arrivato al punto di saturazione per gli improvement dovuti all'aggiustamento dei parametri. Ora per ottenere ulteriori miglioramenti posso solo migliorare modello e/o feature.
Una feature che posso creare per provare a migliorare il mio score è: superficie/stanze; prima dell'aggiunta arrivo a 35.38, a seguito dell'aggiunta: 35.29.

In [ ]:
import joblib

joblib.dump(grid_search.best_estimator_, "HousePredictionRegressor.pkl")

['HousePredictionRegressor.pkl']